In [1]:
import numpy as np
import random

In [2]:
exploration_start = []
action_space = ['hit','stick']
value_dict, total_value, total_visits = {},{},{}
for player_sum in range(2,22):
    for dealer_card in range(2,12):
        for action in action_space:
            exploration_start.append([player_sum,dealer_card,action])
            value_dict[player_sum,dealer_card,action] = 0
            total_value[player_sum,dealer_card,action] = 0
            total_visits[player_sum,dealer_card,action] = 0

In [3]:
def draw_card():
    card_pool = [i for i in range(2,12)] + [10, 10, 10]
    return np.random.choice(card_pool)

def dealer_operation(dealer_value):
    if dealer_value > 21:
        return 'blow up'
    if dealer_value >= 18:
        return dealer_value
    if dealer_value < 18:
        return dealer_operation(dealer_value + draw_card())

def my_strategy(player_value,dealer_value):
    global best_action
    if player_value > 21:
        return 'blow up'
    if value_dict[player_value,dealer_value,'stick'] > value_dict[player_value,dealer_value,'hit']:
        best_action = 'stick'
    elif value_dict[player_value,dealer_value,'stick'] <= value_dict[player_value,dealer_value,'hit']:
        best_action = 'hit'
    if best_action == 'hit':
        visited_states_action_pairs.append([player_value,dealer_value,'hit'])
        return my_strategy(player_value + draw_card(),dealer_value)
    elif best_action == 'stick':
        visited_states_action_pairs.append([player_value,dealer_value,'stick'])
        return player_value

def winner(player_value,dealer_value):
    global result
    if player_value == 'blow up':
        return -1
    if dealer_value == 'blow up':
        return 1
    if player_value > dealer_value:
        return 1
    if player_value == dealer_value:
        return 0
    if player_value < dealer_value:
        return -1

In [4]:
for i in range(500000):
    start = random.choice(exploration_start)
    #print(start)
    visited_states_action_pairs = [start]
    action = start[-1]
    if action == 'hit':
        player_value = start[0] + draw_card()
        player_final = my_strategy(player_value,start[1])
    elif action == 'stick':
        player_final = start[0]
    episode_result = winner(player_final, dealer_operation(start[1]))
    #print(visited_states_action_pairs)
    for i in visited_states_action_pairs:
        total_visits[i[0],i[1],i[2]] += 1
        total_value[i[0],i[1],i[2]] += episode_result
        value_dict[i[0],i[1],i[2]] = round(total_value[i[0],i[1],i[2]] / total_visits[i[0],i[1],i[2]], 3)

#print(value_dict)
print(my_strategy(2,3))

12


In [5]:
policy_dict,unreasonable_stick_sum, unreasonable_hit_sum = {},0,0
for player_sum in range(2,22):
    for dealer_card in range(2,12):
        if value_dict[player_sum,dealer_card,'hit'] > value_dict[player_sum,dealer_card,'stick']:
            policy_dict[player_sum,dealer_card] = 'hit'
            if player_sum >= 16:
                unreasonable_hit_sum += 1
        else:
            policy_dict[player_sum,dealer_card] = 'stick'
            if player_sum <= 10:
                unreasonable_stick_sum += 1

#print(policy_dict)
print('Number of unreasonable hits at high player value: ',unreasonable_hit_sum)
print('Number of unreasonable sticks at low player value: ',unreasonable_stick_sum)


Number of unreasonable hits at high player value:  0
Number of unreasonable sticks at low player value:  3
